In [ ]:
import requests, threading, os, time, pyautogui, csv, pytesseract

import pandas as pd
from pdf2image import convert_from_path
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


In [ ]:
shutdown_flag = threading.Event()
def get_hrefs(urls = ['https://www.codot.gov/business/bidding/bid-tab-archives']): #Possibility of collecting also for other years.
    hrefs = []
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'lxml')
        main_content = soup.find('div',{'id' : 'content-core'})
        table =  main_content.find('table')
        for a in table.findAll('a', href=True):
            document_id = a['href'].split('=')[-1]
            if document_id not in [doc.split('=')[-1] for doc in hrefs]: #Avoiding duplication when extracting documents from multiple URLs. 
                hrefs.append(a['href'])
    return(hrefs)

# Split the hrefs list into chunks
def chunk_list(data, num_chunks):
    chunk_size = len(data) // num_chunks
    remainder = len(data) % num_chunks
    chunks = []
    start = 0
    for i in range(num_chunks):
        end = start + chunk_size + (1 if i < remainder else 0)
        chunks.append(data[start:end])
        start = end
    return chunks

# Reuse the setup_driver function
def setup_driver(download_directory, headless=True):
    chrome_options = Options()
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.90 Safari/537.36"
    )
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_directory,
        "download.prompt_for_download": False,
        "i": True,
    })
    service = Service(os.path.join(os.getcwd(), "Chromedriver/chromedriver"))
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# Function to click at specified coordinates
def click_at_coordinates(driver, x, y):
    try:
        ActionChains(driver).move_by_offset(x, y).click().perform()
        ActionChains(driver).move_by_offset(-x, -y).perform()  # Reset to original position
    except Exception as e:
        print(f"Error performing click at coordinates ({x}, {y}): {e}")

# Function to download PDFs for a worker's chunk
def worker_process(chunk, driver, window_x, window_y, SLEEP=7):
    for href in chunk:
        if shutdown_flag.is_set():
            print(f"Shutdown requested, exiting {href}")
            break

        print(f"Fetching: {href}")
        try:
            driver.get(href)
            time.sleep(SLEEP)

            window_position = driver.get_window_position()
            browser_x = window_position['x']
            browser_y = window_position['y']

            window_size = driver.get_window_size()
            viewport_size = driver.execute_script("return {width: window.innerWidth, height: window.innerHeight};")
            toolbar_height = window_size['height'] - viewport_size['height']

            # Calculate the position inside the window
            ChromeDriver_X = window_x - browser_x
            ChromeDriver_Y = window_y - browser_y - toolbar_height

            click_at_coordinates(driver, ChromeDriver_X, ChromeDriver_Y)
            time.sleep(3)

        except TimeoutException as e:
            print(f"Timeout while processing: {href}. Skipping...")
        except Exception as e:
            print(f"Error occurred with {href}: {e}")

# Multi-threaded function to get PDFs
def get_pdfs_multithread(hrefs, window_x, window_y, max_threads=4, SLEEP=8):
    # Split hrefs into chunks for each worker
    chunks = chunk_list(hrefs, max_threads)
    drivers = [setup_driver(os.path.join(os.getcwd(), "Bidding_Data")) for i in range(max_threads)]

    try:
        with ThreadPoolExecutor(max_threads) as executor:
            # Submit each chunk and its corresponding WebDriver instance
            for chunk, driver in zip(chunks, drivers):
                executor.submit(worker_process, chunk, driver, window_x, window_y, SLEEP)
    except KeyboardInterrupt:
        print("KeyboardInterrupt detected, stopping threads...")
        shutdown_flag.set()  # Notify threads to stop
    finally:
        # Quit all WebDriver instances
        for driver in drivers:
            driver.quit()

def read_pdfs(datafolder, outfile = 'data.csv', split_val = '-', index = 3):
     with open(outfile, 'a', newline='', encoding='utf-8') as c:
        writer = csv.writer(c)
        writer.writerow(['Auction','Num Bids', 'Eng', 'Lowest Bid'] + [f'{i}_bid' for i in range(2, 11)])
        cols = ["Rank", "Vendor Name", "Total Bid"]
        bounding_box = (297, 1290, 2985, 2472)
        files = os.listdir(datafolder)
    
        for file in files:
            pdffile = os.path.join(datafolder, file)
            auction_id = file.split(split_val)[index]
            print(f'Opening {auction_id}')
        
            image = convert_from_path(pdffile, dpi=300, first_page=2, last_page=2)
            try:
                cropped_img = image[0].crop(bounding_box)
            except IndexError as e:
                print(e)
                continue
            raw_text = pytesseract.image_to_string(cropped_img, config="--psm 6")
            rows = raw_text.split("\n")
            table_data = [row.split() for row in rows if row.strip()]
        
            cleaned_data = []
            for row in table_data:
                if len(row) < 3:
                    continue 
                row.pop(1)
                row = row[:-2]
                numeric_idx = next((i for i, val in enumerate(row) if val.replace('.', '', 1).isdigit()), None)
                if numeric_idx is None:
                    continue
                vendor_name = []
                for i in range(numeric_idx + 1, len(row)):
                    if row[i].startswith('$'):
                        dollar_idx = i
                        break
                    vendor_name.append(row[i])
                else:
                    continue 
                vendor_name = " ".join(vendor_name)
                total_bid = "".join(row[dollar_idx:])
                cleaned_row = row[:numeric_idx + 1] + [vendor_name, total_bid]
                cleaned_data.append(cleaned_row)
            try:
                df = pd.DataFrame(cleaned_data, columns=cols)
                df['Total Bid']= df['Total Bid'].str.replace('[\$,]', '', regex=True).astype(float)
                num_bids = df['Rank'].astype(int).max()
                Est = df['Total Bid'][df['Rank'] == '0'].item()
                winning_bid = df['Total Bid'][df['Rank'] == "1"].item()
                if num_bids > 1:
                    ranks = df['Rank'].to_numpy()[(df['Rank'] != '0') & (df['Rank'] != '1')]
                    other_bids = [df['Total Bid'][df['Rank'] == rank].item() for rank in ranks]
            except ValueError as e:
                print(e)
                continue
            print('----------------------------------------------------------------')
            print(f'Auction id: {auction_id}\nNumber of bids: {num_bids}\nWinning bid: {winning_bid}\nOther bids: {other_bids}\nEstimate: {Est}')
            print('----------------------------------------------------------------')
            row = [auction_id, num_bids, Est, winning_bid] + other_bids
            writer.writerow(row)
            other_bids = []

    

In [ ]:
hrefs = get_hrefs(urls=['https://www.codot.gov/business/bidding/bid-tab-archives',
                        'https://www.codot.gov/business/bidding/bid-tab-archives/2023-bid-tabs/bid-tabs-2023',
                         'https://www.codot.gov/business/bidding/bid-tab-archives/2022-bid-tabs/copy_of_bid-tabs-2022'
                         ])

In [ ]:
#CALIBRATION
print('INSTRUCTIONS\n-----------------------------')
time.sleep(2)
print('The window with the PDF will open in chrome browser. Please hover your mouse on the OPEN button once prompted to do so.')
time.sleep(2)
print('The window will close once the process is complete.')
time.sleep(2)
print('Opening window....')
directory = os.path.join(os.getcwd(), "Bidding_Data")
if not os.path.exists(directory):
        os.makedirs(directory)
driver = setup_driver(directory, headless=False)
driver.get(hrefs[0])
time.sleep(7)
print("Move your mouse to the button. Position will be captured in 5 seconds...")
time.sleep(5)
x, y = pyautogui.position()
print(f"Mouse position: ({x}, {y})")
driver.quit()

In [ ]:
#Download all the pdfs.
get_pdfs_multithread(hrefs, x, y, max_threads=10,SLEEP=8) #Choose threads sensibly.

In [3]:
#Set the path to the pdf folder.
pdf_folder = 'Bidding_Data'
dir = os.path.join(os.getcwd(), pdf_folder)

In [4]:
#Retrieve the vendor ranking table from pdfs
read_pdfs(dir, outfile = 'bids1.csv')

Opening  26055 
----------------------------------------------------------------
Auction id:  26055 
Number of bids: 2
Winning bid: 1066564.5
Other bids: [1231284.0]
Estimate: 1162614.0
----------------------------------------------------------------
Opening  20912 
----------------------------------------------------------------
Auction id:  20912 
Number of bids: 7
Winning bid: 1747562.65
Other bids: [1870875.3, 1878649.35, 1889633.0, 2391868.0, 2397750.0, 3325324.0]
Estimate: 1800000.0
----------------------------------------------------------------
Opening  24881 
----------------------------------------------------------------
Auction id:  24881 
Number of bids: 2
Winning bid: 2924208.0
Other bids: [3710384.2]
Estimate: 2536283.06
----------------------------------------------------------------
Opening  24921 
----------------------------------------------------------------
Auction id:  24921 
Number of bids: 3
Winning bid: 2895069.0
Other bids: [2988158.0, 3898085.16]
Estimate: 2

In [5]:
bids_df = pd.read_csv('bids1.csv')
bids_df.head()

,Auction,Num Bids,Eng,Lowest Bid,2_bid,3_bid,4_bid,5_bid,6_bid,7_bid,8_bid,9_bid,10_bid
0,26055,2,1162614.00,1066564.50,1231284.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20912,7,1800000.00,1747562.65,1870875.3,1878649.35,1889633.00,2391868.00,2397750.0,3325324.0,NaN,NaN,NaN
2,24881,2,2536283.06,2924208.00,3710384.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24921,3,2800000.01,2895069.00,2988158.0,3898085.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24063,5,4626079.20,3618938.30,3751213.5,3896707.40,4150364.02,4766033.32,NaN,NaN,NaN,NaN,NaN


In [6]:
#Retrieve the vendor ranking table from old pdfs
outfile = 'bids2.csv'
dir2 = os.path.join(os.getcwd(), 'pdf_CDT')
read_pdfs(dir2, outfile=outfile, index = 0)
bids_df_old = pd.read_csv(outfile)
bids_df_old.head()

Opening 22437
----------------------------------------------------------------
Auction id: 22437
Number of bids: 5
Winning bid: 1068716.0
Other bids: [1076637.7, 1085869.0, 1168705.0, 1414966.0]
Estimate: 768693.0
----------------------------------------------------------------
Opening 23585
----------------------------------------------------------------
Auction id: 23585
Number of bids: 4
Winning bid: 2154630.7
Other bids: [2272702.31, 2385128.51, 2656006.72]
Estimate: 2200255.3
----------------------------------------------------------------
Opening 23485
----------------------------------------------------------------
Auction id: 23485
Number of bids: 7
Winning bid: 4942712.29
Other bids: [5013765.31, 5078192.25, 5352263.45, 5632142.0, 7358759.0, 9075974.0]
Estimate: 6204360.5
----------------------------------------------------------------
Opening 23112
----------------------------------------------------------------
Auction id: 23112
Number of bids: 2
Winning bid: 1832965.0
Other

,Auction,Num Bids,Eng,Lowest Bid,2_bid,3_bid,4_bid,5_bid,6_bid,7_bid,8_bid,9_bid,10_bid
0,22437,5,768693.00,1068716.00,1076637.70,1085869.00,1168705.00,1414966.0,NaN,NaN,NaN,NaN,NaN
1,23585,4,2200255.30,2154630.70,2272702.31,2385128.51,2656006.72,NaN,NaN,NaN,NaN,NaN,NaN
2,23485,7,6204360.50,4942712.29,5013765.31,5078192.25,5352263.45,5632142.0,7358759.00,9075974.00,NaN,NaN,NaN
3,23112,2,1550097.00,1832965.00,2071793.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23186,7,2224481.35,1710210.16,1798693.64,2054579.91,2054960.00,2120840.0,2147241.25,3121062.66,NaN,NaN,NaN


In [7]:
#Merging new and old dataframes based on auction id.
overlap = set(bids_df['Auction']).intersection(bids_df_old['Auction'])
df_unique = bids_df_old[~bids_df_old['Auction'].isin(overlap)]
merged_df = pd.concat([bids_df, df_unique], ignore_index=True)
merged_df.to_csv('bids_final.csv')
merged_df = merged_df.dropna(axis=1, how='all')
merged_df.head()

,Auction,Num Bids,Eng,Lowest Bid,2_bid,3_bid,4_bid,5_bid,6_bid,7_bid,8_bid,9_bid,10_bid
0,26055,2,1162614.00,1066564.50,1231284.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20912,7,1800000.00,1747562.65,1870875.3,1878649.35,1889633.00,2391868.00,2397750.0,3325324.0,NaN,NaN,NaN
2,24881,2,2536283.06,2924208.00,3710384.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24921,3,2800000.01,2895069.00,2988158.0,3898085.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24063,5,4626079.20,3618938.30,3751213.5,3896707.40,4150364.02,4766033.32,NaN,NaN,NaN,NaN,NaN
